In [1]:
import syft as sy
from syft.core.node.common.node_service.tff.tff_messages import TFFMessageWithReply
from syft.core.node.common.node_service.ping.ping_messages import PingMessageWithReply
from syft.grid import GridURL
from syft.core.node.common.node_service.simple.simple_messages import NodeRunnableMessageWithReply
from syft.core.node.common.node_service.get_repr.get_repr_messages import GetReprMessage

/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into node... done!


In [3]:
!pip install --quiet --upgrade --user nest-asyncio

In [4]:
# import nest_asyncio
# nest_asyncio.apply()

In [5]:
# msg = NodeRunnableMessageWithReply("Hello")
msg = TFFMessageWithReply("Hello")
# sy.serialize(msg)
# print(msg)
# serial_msg = sy.serialize(msg, to_bytes=True)
# new_msg = sy.deserialize(serial_msg, from_bytes=True)
# print(new_msg.stuff)
# print(sy.deserialize(sy.serialize(msg, to_bytes=True),from_bytes=True)==msg)
# print(type(domain))
# print(dir(domain))
reply_msg = domain.send_immediate_msg_with_reply(msg)

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'payload', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.tff.tff_messages.TFFReplyMessage'>


In [6]:
reply_msg.payload

'Hello from TFF Service...Hello'

In [7]:
from syft import serialize
msg = PingMessageWithReply("Hello")

serialize(msg, to_bytes=True)

b'\nJsyft.core.node.common.node_service.ping.ping_messages.PingMessageWithReply\x12\x8d\x01\n\x06kwargs\x127\n\x16syft.lib.python.String\x12\x1d\n\x05Hello\x12\x12\n\x10\x1dEW\xf9\xd0\xc7A\xb1\xb8];\xedkL\x9d\x91\x18\x01\x1aJsyft.core.node.common.node_service.ping.ping_messages.PingMessageWithReply'

In [8]:
import syft as sy
dir(sy)

['Any',
 'Device',
 'DeviceClient',
 'DistributionNotFound',
 'Domain',
 'DomainClient',
 'Network',
 'NetworkClient',
 'NetworkRegistry',
 'Path',
 'ReprMessage',
 'Tensor',
 'VirtualMachine',
 'VirtualMachineClient',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_networks',
 'ast',
 'autodp',
 'autograd',
 'common',
 'connect',
 'core',
 'deserialize',
 'experimental_flags',
 'flags',
 'get_distribution',
 'grid',
 'lib',
 'lib_ast',
 'load',
 'load_lib',
 'logger',
 'login',
 'module_property',
 'proto',
 'proxy',
 'register',
 'registry',
 'row_entity_phi',
 'serialize',
 'single_entity_phi',
 'sys',
 'util',
 'wrappers']

In [9]:
from syft.core.node.common.action.tff_action import TFFAction

In [10]:
msg = TFFAction(address=domain.address)
domain.send_immediate_msg_without_reply(msg)


In [11]:
ser_msg = sy.serialize(msg) 
print(type(ser_msg))
de_msg = sy.deserialize(ser_msg)
print(type(de_msg))

<class 'proto.core.node.common.action.tff_action_pb2.TFFAction'>
<class 'syft.core.node.common.action.tff_action.TFFAction'>
